## All you need to do for this to work is:
1. download 'Group Data.csv' and 'Step Data.csv' into the same location as this script
2. Run all cells
3. Files will be outputted into same directory as this script

### Things to Improve
- group's timeToComplete should be automatic based on steps
- No included functionality for helpUrlLabel at the moment

In [1]:
import pandas as pd
import json
import numpy as np

## Pulling in the data from CSV

In [2]:
# Pull in group data and fill in NaN values with ""
group_data = pd.read_csv('Group Data.csv')
group_data = group_data.fillna("")
print(group_data)

   Group ID                                          groupName  \
0         1            Get Started with Appointment Management   
1         2      Setup Work Type and Work Type Code Set Bundle   
2         3  Set Up Identifiers to Link the External Schedu...   
3         4     Setup Configuration for Appointment Scheduling   
4         5  Setup Appointment Management with Salesforce S...   
5         6           Verify Appointment Management is Working   
6         7  Setup Einstein Discovery for Appointment Manag...   

             groupDescription  timeToComplete  groupOrder  
0  Setup provider data models               9           1  
1                                           5           2  
2                                           4           3  
3                                           3           4  
4                                           1           5  
5                                           3           6  
6                                           1      

In [3]:
# Attach a steps field to each group data
group_data["steps"] = [[] for i in range(len(group_data))]

In [4]:
# Read in Step Data and fill NaN values
step_data = pd.read_csv("Step Data.csv")
step_data = step_data.fillna("")

## Convert pulled in data into dictionaries by ID

In [5]:
# Attach an action field to each step data
step_data["action"] = [None for i in range(len(step_data))]

In [6]:
step_data["dependency"][1]

''

In [7]:
# set dependencies to blank if nothing passed in
temp = step_data["dependency"]
for i in range(len(temp)):
    if temp[i]=="":
        temp[i] = []
        
print(step_data["dependency"])

0     []
1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
20    []
21    []
22    []
23    []
24    []
25    []
Name: dependency, dtype: object


<ipython-input-7-510df97db923>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i] = []


In [8]:
# Covert each row into a hashmap/dictionary
group_dicts_list = group_data.to_dict(orient="records")
step_data_list = step_data.to_dict(orient="records")

## Converting names and descriptions into xml
All of the things that are xml variables:
1. groupName
2. groupDescription
3. step->name
4. step->description
5. action->name
6. Title
7. Header
8. DescriptionForTurnoff
9. DescriptionForTurnOn
10. Tottle Title
11. Toggle Message
- Will also be updating the data to have variable names instead of text

In [9]:
# Stuff to add at the beginning that's not being dynamically created
preexisting = \
"""
    <param name="Header">Enable Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnon">Turn on Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnoff">Turn off Patient Appointment Scheduling</param>
    <param name="ProviderSearchOffError">To enable Patient Appointment Scheduling, first enable Provider Search.</param>
    <param name="ToggleTitle">Patient Appointment Scheduling was disabled</param>
    <param name="ToggleMessage">Patient Appointment Scheduling must be enabled to schedule appointments.</param>
"""

In [10]:
# Creating title and feature dictionary
title = "Patient Appointment Scheduling"
featureNameStarter = {}
featureName = "PatientAppointmentScheduling.Title"
titleXML = '<param name="Title">{title}</param>'.format(title=title)
featureJSON = {}
featureJSON["featureName"] = featureName
featureJSON["configurations"] = []
print(featureJSON)
basicSetupXML = titleXML + preexisting
print(basicSetupXML)

{'featureName': 'PatientAppointmentScheduling.Title', 'configurations': []}
<param name="Title">Patient Appointment Scheduling</param>
    <param name="Header">Enable Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnon">Turn on Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnoff">Turn off Patient Appointment Scheduling</param>
    <param name="ProviderSearchOffError">To enable Patient Appointment Scheduling, first enable Provider Search.</param>
    <param name="ToggleTitle">Patient Appointment Scheduling was disabled</param>
    <param name="ToggleMessage">Patient Appointment Scheduling must be enabled to schedule appointments.</param>



In [11]:
# Choosing the name of the parent XML
section_name = "PatientAppointmentSchedulingSetupPage"
section_xml_start = '<section name="{section_name}">'.format(section_name=section_name)
section_xml_end = '</section>'

In [12]:
# Creating params for group name and saving in a dictionary
group_params = []
for row in group_dicts_list:
    # Group Names
    param_name = "Group" + row['groupName'].title().replace(" ", "")
    display_name = row['groupName']
    new_param = '<param name="{param_name}">{display_name}</param>'.format(param_name=param_name, display_name=display_name)
    group_params.append(new_param)
    # Updating the row to store the variable
    row['groupName'] = section_name + '.' + param_name
    
    # Group Description
    if row['groupDescription'] != "":
        param_name = param_name + "Desc"
        display_name = row['groupDescription']
        new_param = '<param name="{param_name}">{display_name}</param>'.format(param_name=param_name, display_name=display_name)
        group_params.append(new_param)
        # Updating the row to store the variable
        row['groupDescription'] = section_name + '.' + param_name

In [13]:
# Creating params for step name and saving in a dictionary
step_params = []
for row in step_data_list:
    # Step Names
    param_name = "Step" + row['name'].title().replace(" ", "")
    display_name = row['name']
    new_param = '<param name="{param_name}">{display_name}</param>'.format(param_name=param_name, display_name=display_name)
    step_params.append(new_param)
    # Updating the row to store the variable
    row['name'] = section_name + '.' + param_name
    
    # Step Description
    if row['description'] != "":
        param_name = param_name + "Desc"
        display_name = row['description']
        new_param = '<param name="{param_name}">{display_name}</param>'.format(param_name=param_name, display_name=display_name)
        step_params.append(new_param)
        # Updating the row to store the variable
        row['description'] = section_name + '.' + param_name
    
    # Action Name
    if row['action_name'] != "":
        param_name = "Action" + row['action_name'].title().replace(" ", "")
        display_name = row['action_name']
        new_param = '<param name="{param_name}">{display_name}</param>'.format(param_name=param_name, display_name=display_name)
        step_params.append(new_param)
        # Updating the row to store the variable
        row['action_name'] = section_name + '.' + param_name


## Converting lists of dictionaries into dictionaries of dictionaries by ID

In [14]:
# # Fixing ordering of groups dictionary
group_order = ["Group ID", "groupName", "groupDescription", "groupOrder", "timeToComplete", "steps"]
for i in range(len(group_dicts_list)):
    temp = {}
    for member in group_order:
        temp[member] = group_dicts_list[i][member]
    group_dicts_list[i] = temp

In [15]:
# # Covert each row into a hashmap/dictionary

# Putting placeholder text into empty descriptions
for i in range(len(group_dicts_list)):
    if group_dicts_list[i]['groupDescription'] == "":
        # Putting in placeholder text
        group_dicts_list[i]['groupDescription'] = "PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc"
        
# store group_dicts_list in a set of dictionaries by group ID
group_dicts = {}
for member in group_dicts_list:
    temp = member.copy()
    new_id = temp.pop("Group ID", None)
    group_dicts[new_id] = temp 
print(group_dicts_list)

[{'Group ID': 1, 'groupName': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagement', 'groupDescription': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc', 'groupOrder': 1, 'timeToComplete': 9, 'steps': []}, {'Group ID': 2, 'groupName': 'PatientAppointmentSchedulingSetupPage.GroupSetupWorkTypeAndWorkTypeCodeSetBundle', 'groupDescription': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc', 'groupOrder': 2, 'timeToComplete': 5, 'steps': []}, {'Group ID': 3, 'groupName': 'PatientAppointmentSchedulingSetupPage.GroupSetUpIdentifiersToLinkTheExternalSchedulingSystem', 'groupDescription': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc', 'groupOrder': 3, 'timeToComplete': 4, 'steps': []}, {'Group ID': 4, 'groupName': 'PatientAppointmentSchedulingSetupPage.GroupSetupConfigurationForAppointmentScheduling', 'groupDescription': 'PatientAppointmentSchedulingSetupPage

## Restructuring action into map within step

In [16]:
# Convert each step's action_ objects into an action map within the map
for step in step_data_list:
    current = step
    action = {}
    action["name"] = current.pop("action_name", "")
    action["type"] = current.pop("action_type", "")
    action["resource"] = current.pop("action_resource", "")
    step["action"] = action

In [17]:
# Fixing Ordering of step dictionary for parser
step_order = ["Group ID", "name", "description", "order", "dependency", "action", "helpUrl", "preValidation", "validationMessage", "timeToComplete"]
for i in range(len(step_data_list)):
    temp = {}
    for member in step_order:
        temp[member] = step_data_list[i][member]
    step_data_list[i] = temp

In [18]:
step_data_list

[{'Group ID': 1,
  'name': 'PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSet',
  'description': 'PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSetDesc',
  'order': 1,
  'dependency': [],
  'action': {'name': 'PatientAppointmentSchedulingSetupPage.ActionUsers',
   'type': 'UrlNavigate',
   'resource': ''},
  'helpUrl': '',
  'preValidation': '',
  'validationMessage': '',
  'timeToComplete': 1},
 {'Group ID': 1,
  'name': 'PatientAppointmentSchedulingSetupPage.StepSetUpDataForAppointmentManagement-VerifyTheProviderDataModel',
  'description': 'PatientAppointmentSchedulingSetupPage.StepSetUpDataForAppointmentManagement-VerifyTheProviderDataModelDesc',
  'order': 2,
  'dependency': [],
  'action': {'name': 'PatientAppointmentSchedulingSetupPage.ActionProviderSearchSettings',
   'type': 'UrlNavigate',
   'resource': ''},
  'helpUrl': '',
  'preValidation': '',
  'validationMessage': '',
  'timeToComplete': 1},
 {'Grou

In [19]:
# Consolidate each step dictionary into a map with key being group ID
step_dicts = {}
for step in step_data_list:
    temp = step.copy()
    new_id = temp.pop("Group ID", None)
    if new_id in step_dicts:
        current = step_dicts[new_id]
        current.append(temp)
    else:
        step_dicts[new_id] = [temp]

## Add step dictionaries into parent group dictionaries

In [20]:
# Add steps to each group
for group_id in group_dicts:
    associated_steps = step_dicts[group_id]
    group_dicts[group_id]["steps"] = associated_steps
print(group_dicts)

{1: {'groupName': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagement', 'groupDescription': 'PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc', 'groupOrder': 1, 'timeToComplete': 9, 'steps': [{'name': 'PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSet', 'description': 'PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSetDesc', 'order': 1, 'dependency': [], 'action': {'name': 'PatientAppointmentSchedulingSetupPage.ActionUsers', 'type': 'UrlNavigate', 'resource': ''}, 'helpUrl': '', 'preValidation': '', 'validationMessage': '', 'timeToComplete': 1}, {'name': 'PatientAppointmentSchedulingSetupPage.StepSetUpDataForAppointmentManagement-VerifyTheProviderDataModel', 'description': 'PatientAppointmentSchedulingSetupPage.StepSetUpDataForAppointmentManagement-VerifyTheProviderDataModelDesc', 'order': 2, 'dependency': [], 'action': {'name': 'PatientAppointmentSchedul

### Creating the featureName and heading setup

In [21]:
# Convert dictionary of group dictionaries into a list of group dictionaries
final_group_lists = []
for group_id in group_dicts:
    final_group_lists.append(group_dicts[group_id])

In [22]:
# putting in groups into the configurations
featureJSON["configurations"] = final_group_lists

In [23]:
json_output = json.dumps(featureJSON, indent=2)
print(json_output)

{
  "featureName": "PatientAppointmentScheduling.Title",
  "configurations": [
    {
      "groupName": "PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagement",
      "groupDescription": "PatientAppointmentSchedulingSetupPage.GroupGetStartedWithAppointmentManagementDesc",
      "groupOrder": 1,
      "timeToComplete": 9,
      "steps": [
        {
          "name": "PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSet",
          "description": "PatientAppointmentSchedulingSetupPage.StepAssignTheHealthCloudFoundationPermissionSetDesc",
          "order": 1,
          "dependency": [],
          "action": {
            "name": "PatientAppointmentSchedulingSetupPage.ActionUsers",
            "type": "UrlNavigate",
            "resource": ""
          },
          "helpUrl": "",
          "preValidation": "",
          "validationMessage": "",
          "timeToComplete": 1
        },
        {
          "name": "PatientAppointmentS

### Config File XML

In [24]:
json_file_name = 'PatientAppointmentScheduling.json'
xml_file_name = 'PatientAppointmentScheduling.xml'
configXML = '<param name="PatientAppointmentSchedulingConfigFile" dnt="true">{file_name}</param>'.format(file_name=json_file_name)

## Create JSON and XML Files

In [25]:
# Write json output to a file
with open(json_file_name, 'w') as outfile:
    outfile.write(json_output)
all_params = ""
#Write XML output
with open(xml_file_name, "w") as outfile:
    all_params = section_xml_start + '\n\t'
    all_params += basicSetupXML
    all_params += '\n\t'.join(group_params) +'\n\t'.join(step_params) 
    all_params += '\n\t' + configXML
    all_params += '\n' + section_xml_end
    outfile.write(all_params)

In [26]:
print(all_params)

<section name="PatientAppointmentSchedulingSetupPage">
	<param name="Title">Patient Appointment Scheduling</param>
    <param name="Header">Enable Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnon">Turn on Patient Appointment Scheduling</param>
    <param name="DescriptionForTurnoff">Turn off Patient Appointment Scheduling</param>
    <param name="ProviderSearchOffError">To enable Patient Appointment Scheduling, first enable Provider Search.</param>
    <param name="ToggleTitle">Patient Appointment Scheduling was disabled</param>
    <param name="ToggleMessage">Patient Appointment Scheduling must be enabled to schedule appointments.</param>
<param name="GroupGetStartedWithAppointmentManagement">Get Started with Appointment Management</param>
	<param name="GroupGetStartedWithAppointmentManagementDesc">Setup provider data models</param>
	<param name="GroupSetupWorkTypeAndWorkTypeCodeSetBundle">Setup Work Type and Work Type Code Set Bundle</param>
	<param name="